In [37]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hktheone\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df=pd.read_csv("movies.csv",encoding = "ISO-8859-1")
df

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...
1,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...
2,3,Inception,2010,8.8,74.0,"Action, Adventure, Sci-Fi",1760209,Christopher Nolan,148,292.58,A thief who steals corporate secrets through t...
3,4,Fight Club,1999,8.8,66.0,Drama,1609459,David Fincher,139,37.03,An insomniac office worker and a devil-may-car...
4,5,Pulp Fiction,1994,8.9,94.0,"Crime, Drama",1570194,Quentin Tarantino,154,107.93,"The lives of two mob hitmen, a boxer, a gangst..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,When Night Is Falling,1995,6.6,NaN,"Drama, Romance",6022,Patricia Rozema,94,1.19,"An uptight and conservative woman, working on ..."
9996,9997,Stachka,1925,7.6,NaN,Drama,6021,Sergei M. Eisenstein,95,NaN,A group of oppressed factory workers go on str...
9997,9998,November Criminals,2017,5.3,31.0,"Crime, Drama, Mystery",6020,Sacha Gervasi,85,NaN,"After his friend is murdered, a Washington, D...."
9998,9999,120,2008,7.3,NaN,"Drama, History, War",6018,Ãzhan Eren,114,NaN,"During the Sarikamis Battle, the Ottoman army ..."


In [4]:
#reducing multiple Genres to single which will be the first one (Crime,Drama----->Crime)
l=[]
for x in list(df["Genre"]):
        l.append(x.split(",")[0])
df["Genre"]=l
df

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...
1,2,The Dark Knight,2008,9.0,84.0,Action,1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...
2,3,Inception,2010,8.8,74.0,Action,1760209,Christopher Nolan,148,292.58,A thief who steals corporate secrets through t...
3,4,Fight Club,1999,8.8,66.0,Drama,1609459,David Fincher,139,37.03,An insomniac office worker and a devil-may-car...
4,5,Pulp Fiction,1994,8.9,94.0,Crime,1570194,Quentin Tarantino,154,107.93,"The lives of two mob hitmen, a boxer, a gangst..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,When Night Is Falling,1995,6.6,NaN,Drama,6022,Patricia Rozema,94,1.19,"An uptight and conservative woman, working on ..."
9996,9997,Stachka,1925,7.6,NaN,Drama,6021,Sergei M. Eisenstein,95,NaN,A group of oppressed factory workers go on str...
9997,9998,November Criminals,2017,5.3,31.0,Crime,6020,Sacha Gervasi,85,NaN,"After his friend is murdered, a Washington, D...."
9998,9999,120,2008,7.3,NaN,Drama,6018,Ãzhan Eren,114,NaN,"During the Sarikamis Battle, the Ottoman army ..."


In [5]:
#supported Genres , limit is 40 otherwise dropepd
g=list(df["Genre"].unique())
for x in g:
    if(df[df["Genre"]==x].count()[0]<200):
        df.drop(df[df['Genre']==x].index, inplace = True)
        
g=list(df["Genre"].unique())
for x in g:
    print(x,df[df['Genre']==x].count()[0])

Drama 2081
Action 2094
Crime 791
Adventure 563
Biography 496
Animation 436
Comedy 2729
Horror 570


In [6]:
#making dictionary genres for simplification 
# Assigning a number to each Genre
genre={}
i=1
for x in g:
    genre[x]=i
    i=i+1
genre

{'Drama': 1,
 'Action': 2,
 'Crime': 3,
 'Adventure': 4,
 'Biography': 5,
 'Animation': 6,
 'Comedy': 7,
 'Horror': 8}

In [7]:
#Getting required columns
df=df[["Genre","Description"]]
nl=[]
for x in list(df["Genre"]):
    nl.append(genre[x])
df["Genre"]=nl
df

,Genre,Description
0,1,Two imprisoned men bond over a number of years...
1,2,When the menace known as the Joker emerges fro...
2,2,A thief who steals corporate secrets through t...
3,1,An insomniac office worker and a devil-may-car...
4,3,"The lives of two mob hitmen, a boxer, a gangst..."
...,...,...
9995,1,"An uptight and conservative woman, working on ..."
9996,1,A group of oppressed factory workers go on str...
9997,3,"After his friend is murdered, a Washington, D...."
9998,1,"During the Sarikamis Battle, the Ottoman army ..."


In [8]:
def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [26]:
textFeatures = df['Description'].copy()
#Extracting important words that will be used as features
textFeatures = textFeatures.apply(pre_process)

# saving trained data for future use 
v_input=pd.DataFrame(textFeatures)
v_input.to_csv("vectorizer_input.csv")

In [29]:
#assigning numbers to features
vectorizer = TfidfVectorizer("english",smooth_idf=True,use_idf=True)
features = vectorizer.fit_transform(textFeatures)
mydf= pd.DataFrame(features)
mydf

,0
0,"(0, 3661)\t0.4123496415052951\n (0, 2999)\t..."
1,"(0, 6956)\t0.2586082271445425\n (0, 5162)\t..."
2,"(0, 2532)\t0.29812360039153124\n (0, 8888)\..."
3,"(0, 9176)\t0.4523936591659689\n (0, 12725)\..."
4,"(0, 14683)\t0.2574310094889511\n (0, 13446)..."
...,...
9755,"(0, 8063)\t0.3151061274904066\n (0, 2399)\t..."
9756,"(0, 10690)\t0.4725652533498205\n (0, 11777)..."
9757,"(0, 14273)\t0.4975021982624719\n (0, 3608)\..."
9758,"(0, 11917)\t0.3500736185909139\n (0, 14527)..."


In [30]:
#using smote to balance features
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 101)
X, y = smote.fit_resample(features,df['Genre'])

In [31]:
print("Genre no: \t count") 
for x in list(set(y)):
  print(str(x)+":    \t   ",np.count_nonzero(x==y))

Genre no: 	 count
1:    	    2729
2:    	    2729
3:    	    2729
4:    	    2729
5:    	    2729
6:    	    2729
7:    	    2729
8:    	    2729


In [32]:
features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [51]:
#Training the model
svc = SVC(kernel='linear', gamma=22)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
print(accuracy_score(labels_test,prediction))

# Saving the trained model
filename = 'finalized_model.sav'
pickle.dump(svc, open(filename, 'wb'))

0.8376459812228074


In [40]:
def predict_genre(msg):
    if(len(msg)<100):
        return(" Kindly insert more than 100 characters ")
    msg=pre_process(msg)
    #using same vectorizer object to generate same number of features from given input
    ft=vectorizer.transform([msg])
    n=svc.predict(ft)[0]
    index=list(genre.values()).index(n)
    return list(genre.keys())[index]

In [44]:
# miscellaneous example
msg="a stranger travels into the land of undead where he is not welcomed but he must survive with whatever means taken"
predict_genre(msg)

'Horror'

In [45]:
#Tenet (Action/Sci-fi)
msg="Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyond real time. "
predict_genre(msg)

'Action'

In [46]:
#Mulan(2020) (Action/Adventure)
msg="A girl disguises as a male warrior and joins the imperial army in order to prevent her sick father from being forced to enlist as he has no male heir."
predict_genre(msg)

'Adventure'

In [47]:
#Monster Hunter (Action/Fantasy)
msg="When Lt. Artemis and her loyal soldiers are transported to a new world, they engage in a desperate battle for survival against enormous enemies with incredible powers. Feature film based on the video game by Capcom."
predict_genre(msg)

'Action'

In [48]:
#Hole in the ground(2019) (Horror/Thriller)
msg="One night, Sarah's young son disappears into the woods behind their rural home. When he returns, he looks the same, but his behavior grows increasingly disturbing. Sarah begins to believe that the boy who returned may not be her son at all."
predict_genre(msg)

'Horror'

In [50]:
#Emma (Romance/Drama)
msg="Following the antics of a young woman, Emma Woodhouse, who lives in Georgian- and Regency-era England and occupies herself with matchmaking - in sometimes misguided, often meddlesome fashion- in the lives of her friends and family."
predict_genre(msg)

'Drama'